In [1]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt

config = "bruz"
if config == "ur1":
    sys.path.insert(0, "../")
    siepelDir = "C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2"
elif config == "bruz":
    sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
    siepelDir = "/home/pleroy/DATA/CAMILL/WR15_v2"

import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.ConfigSer import ConfigSer

In [2]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [3]:
cmap='jet'
configFilenameWr15 = os.path.join(siepelDir, "wr15_plaqueSupport.ini")
configFilenameWr10 = os.path.join(siepelDir, "wr10_plaqueSupport.ini")
conf15 = ConfigSer(configFilenameWr15)
conf10 = ConfigSer(configFilenameWr10)
conf15.printConfig()
conf10.printConfig()

iWR15 = 20
iWR10 = 21

vmin = -90
vmax = -40
cmap='jet'

centerFreqWR15 = 62.5
bandWidthWR15 = 12.4
centerFreqWR10 = 92.5
bandWidthWR10 = 17.4

/home/pleroy/DATA/CAMILL/WR15_v2/wr15_plaqueSupport.ini
nb_elev 41
nb_freq 2001
nb_ssb 3
el0 0.7
az0 -0.44
base_path /home/pleroy/DATA/CAMILL/WR15_v2
out_path /home/pleroy/DATA/CAMILL/WR15_v2/processing/wr15
gateWidth 500.0
gateCenter 6450.0
/home/pleroy/DATA/CAMILL/WR15_v2/wr10_plaqueSupport.ini
nb_elev 41
nb_freq 2801
nb_ssb 4
el0 0.65
az0 -0.44
base_path /home/pleroy/DATA/CAMILL/WR15_v2
out_path /home/pleroy/DATA/CAMILL/WR15_v2/processing/wr10
gateWidth 500.0
gateCenter 6450.0


In [4]:
dirWR15 = f"{conf15.out_path}/data"
dirWR10 = f"{conf10.out_path}/data"
prefix = f"{conf10.base_path}/processing/wr15_wr10/"

In [5]:
freq_WR15 = np.load( f"{dirWR15}/freq_WR15.npy" )
freq_WR10 = np.load( f"{dirWR10}/freq_WR10.npy" )
elevation_WR15 = np.load( f"{dirWR15}/elevation_WR15.npy" )
elevation_WR10 = np.load( f"{dirWR10}/elevation_WR10.npy" )

# Compare WR15 / WR10

In [6]:
dictOfMeasurements = {
    "A": (("20200701", "000_A"), ("20200709", "036_A")),
    "C": (("20200701", "001_C"), ("20200709", "037_C")),
    "E": (("20200701", "002_E"), ("20200715", "038_E")),
    "W": (("20200701", "003_W"), ("20200715", "040_W_v2")), # ("20200715", "039_W_u")
    "V": (("20200701", "004_V"), ("20200715", "041_V")),
    "G": (("20200701", "005_G"), ("20200715", "042_G")),
    "L": (("20200703", "006_L"), ("20200715", "043_L")),
    "M": (("20200703", "007_M"), ("20200715", "044_M")),
    "X": (("20200703", "008_X"), ("20200715", "045_X")),
    "H": (("20200703", "009_H"), ("20200716", "051_H")),
    "I": (("20200703", "010_I"), ("20200716", "050_I")),
    "K": (("20200703", "011_K"), ("20200715", "048_K")),
    "J": (("20200703", "012_J"), ("20200716", "049_J")),
    "O": (("20200703", "013_O"), ("20200715", "046_O")),
    "N": (("20200703", "014_N"), ("20200715", "047_N")),
    "D": (("20200703", "015_AA"), ("20200716", "053_AA")),
    "T": (("20200703", "016_Y"), ("20200716", "054_Y")),
    "AB": (("20200703", "017_AB"), ("20200716", "052_AB")),
    "Z": (("20200703", "018_Z"), ("20200716", "055_Z")),
    "AC": (("20200819", "065_AC"), ("20200818", "060_AC")),
    "AD": (("20200819", "066_AD"), ("20200818", "061_AD")),
}

wr15_to_wr10_date = 20200709
wr10_to_wr15_date = 20200818

In [7]:
fig, ax = plt.subplots(1,1)
title = prefix

getMax = 0

for myAbs in dictOfMeasurements["A"]:
    date, absorbant = myAbs
    if int(date) < wr15_to_wr10_date or int(date) > wr10_to_wr15_date:
        # this is a WR15 measurement
        reflectivite = np.load( f"{dirWR15}/reflectivite/{absorbant}_{date}.npy" )
        toPlot = reflectivite[iWR15, :]
        if getMax:
            toPlot = np.amax(np.abs(reflectivite), axis=0)
        ax.plot( freq_WR15, ser.dB( toPlot ), 
                label = f"WR15, {absorbant}, {date}, elevation = {elevation_WR15[iWR15]:.2f}" )
    else:
        # this is a WR10 measurement
        reflectivite = np.load( f"{dirWR10}/reflectivite/{absorbant}_{date}.npy" )  
        toPlot = reflectivite[iWR10, :]
        if getMax:
            toPlot = np.amax(np.abs(reflectivite), axis=0)
        ax.plot( freq_WR10, ser.dB( toPlot ), 
                label = f"WR10, {absorbant}, {date}, elevation = {elevation_WR10[iWR10]:.2f}" )
    if title == prefix:
        if getMax:
            title = prefix + f"max/{absorbant}_{date}"
        else:
            title = prefix + f"{absorbant}_{date}"
    else:
        title = title + f"_{absorbant}_{date}"

ax.legend()
ax.grid()
ax.set_xlabel("[GHz]")
ax.set_ylabel("[dB]")
ax.set_title(f"Réflectivité")
if getMax:
    title += "_max"

# Compare all

In [ ]:
for root, dirs, files in os.walk(prefix):
    for f in files:
        if f.endswith(".png"):
            os.remove(os.path.join(root, f)) # remove or unlink does the same

In [9]:
fig, ax = plt.subplots(1,1)

getMax = 0

for item in dictOfMeasurements.items():
    ref, tests = item
    title = prefix
    fig, ax = plt.subplots(1,1)
    print(f"{ref}, {len(tests)} measurements")
    for test in tests:
        date, absorbant = test
        if int(date) < wr15_to_wr10_date or int(date) > wr10_to_wr15_date:
            # this is a WR15 measurement
            reflectivite = np.load( f"{dirWR15}/reflectivite/{absorbant}_{date}.npy" )
            toPlot = reflectivite[iWR15, :]
            if getMax:
                toPlot = np.amax(np.abs(reflectivite), axis=0)
            ax.plot( freq_WR15, ser.dB( toPlot ), 
                    label = f"WR15, {absorbant}, {date}, elevation = {elevation_WR15[iWR15]:.2f}" )
        else:
            # this is a WR10 measurement
            reflectivite = np.load( f"{dirWR10}/reflectivite/{absorbant}_{date}.npy" )    
            toPlot = reflectivite[iWR10, :]
            if getMax:
                toPlot = np.amax(np.abs(reflectivite), axis=0)
            ax.plot( freq_WR10, ser.dB( toPlot ), 
                    label = f"WR10, {absorbant}, {date}, elevation = {elevation_WR10[iWR10]:.2f}" )
        if getMax:
            title = prefix + f"max/{absorbant}_{date}"
        else:
            title = prefix + f"elev_i/{absorbant}_{date}"
    ax.legend()
    ax.grid()
    ax.set_xlabel("[GHz]")
    ax.set_ylabel("[dB]")
    ax.set_title(f"Réflectivité")
    if getMax:
        title += "_max"
    fig.savefig(f"{title}.png")
    plt.close()

A, 2 measurements
C, 2 measurements
E, 2 measurements
W, 2 measurements
V, 2 measurements
G, 2 measurements
L, 2 measurements
M, 2 measurements
X, 2 measurements
H, 2 measurements
I, 2 measurements
K, 2 measurements
J, 2 measurements
O, 2 measurements
N, 2 measurements
D, 2 measurements
T, 2 measurements
AB, 2 measurements
Z, 2 measurements
AC, 2 measurements
AD, 2 measurements


# Plot cmap

## One

In [ ]:
fig, ax = plt.subplots(1,1)
# wr15
idxWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) < bandWidthWR15 )
idxAltWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) >= bandWidthWR15 )
# wr10
idxWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) < bandWidthWR10 )
idxAltWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) >= bandWidthWR10 )

refAbs = "A"
meas0 = dictOfMeasurements[refAbs][0]
meas1 = dictOfMeasurements[refAbs][-1]
for meas in meas0, meas1:
    date, absorbant = meas
    if int(date) < wr15_to_wr10_date or int(date) > wr10_to_wr15_date:
        # this is a WR15 measurement
        reflectivite = np.load( f"{dirWR15}/reflectivite/{absorbant}_{date}.npy" )
        # drop lower and higher frequencies values
        toPlot = np.zeros(reflectivite.shape)
        toPlot[:,idxWR15]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR15] ) )
        toPlot[:,idxAltWR15] = np.nan
        im = ax.pcolormesh( elevation_WR15, np.flip( freq_WR15 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
    else:
        # this is a WR10 measurement
        reflectivite = np.load( f"{dirWR10}/reflectivite/{absorbant}_{date}.npy" ) 
        # drop lower and higher frequencies values
        toPlot = np.zeros(reflectivite.shape)
        toPlot[:,idxWR10]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR10] ) )
        toPlot[:,idxAltWR10] = np.nan
        im = ax.pcolormesh( elevation_WR10, np.flip( freq_WR10 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
    
title = f"Réflectivité {refAbs}: {meas0[1]} / {meas1[1]}"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )
fig.savefig(f"{prefix}/pcolormesh/reflectivite_{refAbs}.png")

## All

In [ ]:
idxWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) < bandWidthWR15 )
idxAltWR15 = np.where( np.abs( freq_WR15 - centerFreqWR15 ) >= bandWidthWR15 )

idxWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) < bandWidthWR10 )
idxAltWR10 = np.where( np.abs( freq_WR10 - centerFreqWR10 ) >= bandWidthWR10 )

for refAbs in dictOfMeasurements.keys():
    fig, ax = plt.subplots(1,1)
    meas0 = dictOfMeasurements[refAbs][0]
    meas1 = dictOfMeasurements[refAbs][-1]
    for meas in meas0, meas1:
        date, absorbant = meas
        print(f"{absorbant}_{date}")
        if int(date) < wr15_to_wr10_date or int(date) > wr10_to_wr15_date:
            # this is a WR15 measurement
            reflectivite = np.load( f"{dirWR15}/reflectivite/{absorbant}_{date}.npy" )
            # drop lower and higher frequencies values
            toPlot = np.zeros(reflectivite.shape)
            toPlot[:,idxWR15]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR15] ) )
            toPlot[:,idxAltWR15] = np.nan
            im = ax.pcolormesh( elevation_WR15, np.flip( freq_WR15 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
        else:
            # this is a WR10 measurement
            reflectivite = np.load( f"{dirWR10}/reflectivite/{absorbant}_{date}.npy" ) 
            # drop lower and higher frequencies values
            toPlot = np.zeros(reflectivite.shape)
            toPlot[:,idxWR10]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idxWR10] ) )
            toPlot[:,idxAltWR10] = np.nan
            im = ax.pcolormesh( elevation_WR10, np.flip( freq_WR10 ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
    
    title = f"Réflectivité {refAbs}: {meas0[1]} / {meas1[1]}"
    ax.set_title(title)
    ax.set_xlabel( "Elévation [°]" )
    ax.set_ylabel( "Fréquence [GHz]" )
    dia.addColorBar(  im, ax, aspect='auto' )
    fig.savefig(f"{prefix}/pcolormesh/reflectivite_{refAbs}.png")
    plt.close()

# Remove all

In [ ]:
for root, dirs, files in os.walk(f"{prefix}"):
    for f in files:
        if f.endswith(".png") or f.endswith(".npy"):
            os.remove(os.path.join(root, f)) # remove or unlink does the same